In [1]:
import pulp

In [102]:
boxes = [1,2,3]
nb_coins = dict(zip(boxes,[1,2,3]))
coins = [1,2,3,4,5,6]
coin_value = dict(zip(coins,[0.05,0.1,0.2,0.5,1,2]))

In [120]:
# create problem
prob = pulp.LpProblem("coins", pulp.LpMaximize)

# x[c][b] = 1 <=> coin c is selected for box b 
x = pulp.LpVariable.dicts("x",(coins,boxes),cat=pulp.LpBinary)
# y[c] = 1 <=> coin c is selected
y = pulp.LpVariable.dicts("y",coins,cat=pulp.LpBinary)

# average value of each box is 1.5
prob += pulp.lpSum(coin_value[c]*nb_coins[b]*x[c][b] for c in x for b in x[c])==1.5*3, "average_is_1.5"

# in each box, there is exactly one type of coin
for b in boxes:
    prob += pulp.lpSum(x[c][b] for c in x)==1, "unique_coin_value_box_%s"%b

# coins must be different in each box
for c in coins:
    prob += y[c]<=pulp.lpSum(x[c][b] for b in x[c]),"use_coin_value_%s_implies_use_it_in_at_least_one_box"%c
    for b in boxes:
        prob += x[c][b]<=y[c]
prob += pulp.lpSum(y)>=3, "use_3_different_types_coins"

# solution
# prob += x[6][2] >= 1, "box_2_coin_value_5"

In [121]:
#prob

In [122]:
prob.solve()

1

In [123]:
for b in boxes:
    for c in coins:
        if pulp.value(x[c][b])>0.9:
            print("box",b,":",nb_coins[b],"coins of value",coin_value[c],", total value",round(nb_coins[b]*coin_value[c],2))

print("average value in box :",pulp.lpSum(coin_value[c]*nb_coins[b]*pulp.value(x[c][b]) for c in x for b in x[c])/3)

box 1 : 1 coins of value 0.2 , total value 0.2
box 2 : 2 coins of value 2 , total value 4
box 3 : 3 coins of value 0.1 , total value 0.3
average value in box : 1.5
